In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.image import imread
from os import listdir
from keras.preprocessing.image import load_img, img_to_array
from numpy import asarray, savez_compressed, load
from collections import Counter
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight, shuffle
from imblearn.over_sampling import SMOTE
from keras.utils import to_categorical
from keras.applications import resnet50  
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.models import Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
#Function to load saved satellite imagery data
def load_saved_dataset(file):
    data = load(file)
    X, y = data['arr_0'], data['arr_1']
    trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.20)
    print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    return trainX, trainY, testX, testY
  
#function to evaluate model    
def eval(testY, predY):
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testY, predY)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testY, predY, average='macro')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testY, predY, average='macro')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testY, predY, average='macro')
    print('F1 score: %f' % f1)
    cm = confusion_matrix(testY, predY)
    print('Confusion Matrix:\n', cm)

In [3]:
#Total Road Accidents - First Split

In [3]:
#load data
trainX, trainY, testX, testY = load_saved_dataset('total_acc.npz')
print(Counter(trainY), Counter(testY))
#find average number of instances
m = round(sum(Counter(trainY).values())/4)
print('Mean: ', m)
#implement SMOTE on minority classes
trainX = trainX.reshape(trainX.shape[0], -1)
sm = SMOTE({1:m, 2:m, 3:m})
X_sm, Y_sm = sm.fit_sample(trainX, trainY)
print('After SMOTE: ', X_sm.shape, Y_sm.shape, Counter(Y_sm))
#split out majority set from dataset
ds_maj = []
X_tmp = []
Y_tmp = []
for i in range(len(X_sm)):
    target = Y_sm[i]
    var = X_sm[i]
    if target == 0:
        ds_maj.append(var)
    if target == 1:
        X_tmp.append(var)
        Y_tmp.append(1)
    if target == 2:
        X_tmp.append(var)
        Y_tmp.append(2)
    if target == 3:
        X_tmp.append(var)
        Y_tmp.append(3)

X_tmp = np.asarray(X_tmp)
Y_tmp = np.asarray(Y_tmp)
ds_maj = np.asarray(ds_maj)
print('After undersampling Majority: ', X_tmp.shape, Y_tmp.shape, ds_maj.shape)
#undersample majority set to m number of instances
ds_maj = shuffle(ds_maj, n_samples=m)
#generate target set of 0's for undersampled majority set
ds_maj_y = np.repeat(0, len(ds_maj))
#combine datasets
X1_tmp = np.concatenate([X_tmp, ds_maj])
Y1_tmp = np.concatenate([Y_tmp, ds_maj_y])
#shuffle combined dataset
X_sm, Y_sm = shuffle(X1_tmp, Y1_tmp, random_state=1)
print('Final Class Counter: ', Counter(Y_sm))
#prepare data for training
X_sm = X_sm.reshape(X_sm.shape[0], 128, 128, 3)
Y_sm = to_categorical(Y_sm)
testY = to_categorical(testY)
print('Final training dataset: ', X_sm.shape, Y_sm.shape)
print('Final validation dataset: ', testX.shape, testY.shape)

(46283, 128, 128, 3) (46283,) (11571, 128, 128, 3) (11571,)
Counter({0: 43936, 1: 1650, 2: 587, 3: 110}) Counter({0: 11001, 1: 399, 2: 145, 3: 26})
Mean:  11571
After SMOTE:  (78649, 49152) (78649,) Counter({0: 43936, 1: 11571, 2: 11571, 3: 11571})
After undersampling Majority:  (34713, 49152) (34713,) (43936, 49152)
Final Class Counter:  Counter({0: 11571, 1: 11571, 2: 11571, 3: 11571})
Final training dataset:  (46284, 128, 128, 3) (46284, 4)
Final validation dataset:  (11571, 128, 128, 3) (11571, 4)


In [4]:
#train first split
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)
model_dssm1 = Model(inputs=base_model.input, outputs=predictions)

model_dssm1.compile(optimizer=Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics=['acc'])

history1 = model_dssm1.fit(X_sm, Y_sm, validation_data=(testX, testY), epochs= 25, batch_size = 256)

W0805 02:00:03.163785 140709607630592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 02:00:03.183815 140709607630592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 02:00:03.189993 140709607630592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0805 02:00:03.213101 140709607630592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0805 02:00:03.213943

Train on 46284 samples, validate on 11571 samples
Epoch 1/25
46284/46284 [==============================] - 223s 5ms/step - loss: 0.5572 - acc: 0.7609 - val_loss: 0.2744 - val_acc: 0.9118
Epoch 2/25
46284/46284 [==============================] - 204s 4ms/step - loss: 0.1518 - acc: 0.9420 - val_loss: 0.2606 - val_acc: 0.9264
Epoch 3/25
46284/46284 [==============================] - 204s 4ms/step - loss: 0.0542 - acc: 0.9819 - val_loss: 0.3224 - val_acc: 0.9475
Epoch 4/25
46284/46284 [==============================] - 204s 4ms/step - loss: 0.0320 - acc: 0.9893 - val_loss: 0.3829 - val_acc: 0.9183
Epoch 5/25
46284/46284 [==============================] - 204s 4ms/step - loss: 0.0280 - acc: 0.9906 - val_loss: 0.3981 - val_acc: 0.8955
Epoch 6/25
46284/46284 [==============================] - 205s 4ms/step - loss: 0.0237 - acc: 0.9923 - val_loss: 0.4275 - val_acc: 0.9414
Epoch 7/25
46284/46284 [==============================] - 205s 4ms/step - loss: 0.0196 - acc: 0.9932 - val_loss: 0.4643 - 

In [5]:
#model predictions
predictions = model_dssm1.predict(testX)
predY = np.argmax(predictions, axis=1)
testY = np.argmax(testY, axis=1)
#Results #1
eval(testY, predY)

Accuracy: 0.922133
Precision: 0.380151
Recall: 0.379247
F1 score: 0.366522
Confusion Matrix:
 [[10542   332   122     5]
 [  272    81    46     0]
 [   70    29    46     0]
 [    9     6    10     1]]


In [ ]:
#Total Road Accidents - Second Split

In [3]:
#load data
trainX, trainY, testX, testY = load_saved_dataset('total_acc.npz')
print(Counter(trainY), Counter(testY))
#find average number of instances
m = round(sum(Counter(trainY).values())/4)
print('Mean: ', m)
#implement SMOTE on minority classes
trainX = trainX.reshape(trainX.shape[0], -1)
sm = SMOTE({1:m, 2:m, 3:m})
X_sm, Y_sm = sm.fit_sample(trainX, trainY)
print('After SMOTE: ', X_sm.shape, Y_sm.shape, Counter(Y_sm))
#split out majority set from dataset
ds_maj = []
X_tmp = []
Y_tmp = []
for i in range(len(X_sm)):
    target = Y_sm[i]
    var = X_sm[i]
    if target == 0:
        ds_maj.append(var)
    if target == 1:
        X_tmp.append(var)
        Y_tmp.append(1)
    if target == 2:
        X_tmp.append(var)
        Y_tmp.append(2)
    if target == 3:
        X_tmp.append(var)
        Y_tmp.append(3)

X_tmp = np.asarray(X_tmp)
Y_tmp = np.asarray(Y_tmp)
ds_maj = np.asarray(ds_maj)
print('After undersampling Majority: ', X_tmp.shape, Y_tmp.shape, ds_maj.shape)
#undersample majority set to m number of instances
ds_maj = shuffle(ds_maj, n_samples=m)
#generate target set of 0's for undersampled majority set
ds_maj_y = np.repeat(0, len(ds_maj))
#combine datasets
X1_tmp = np.concatenate([X_tmp, ds_maj])
Y1_tmp = np.concatenate([Y_tmp, ds_maj_y])
#shuffle combined dataset
X_sm, Y_sm = shuffle(X1_tmp, Y1_tmp, random_state=1)
print('Final Class Counter: ', Counter(Y_sm))
#prepare data for training
X_sm = X_sm.reshape(X_sm.shape[0], 128, 128, 3)
Y_sm = to_categorical(Y_sm)
testY = to_categorical(testY)
print('Final training dataset: ', X_sm.shape, Y_sm.shape)
print('Final validation dataset: ', testX.shape, testY.shape)

(46283, 128, 128, 3) (46283,) (11571, 128, 128, 3) (11571,)
Counter({0: 43973, 1: 1620, 2: 582, 3: 108}) Counter({0: 10964, 1: 429, 2: 150, 3: 28})
Mean:  11571
After SMOTE:  (78686, 49152) (78686,) Counter({0: 43973, 1: 11571, 2: 11571, 3: 11571})
After undersampling Majority:  (34713, 49152) (34713,) (43973, 49152)
Final Class Counter:  Counter({0: 11571, 1: 11571, 2: 11571, 3: 11571})
Final training dataset:  (46284, 128, 128, 3) (46284, 4)
Final validation dataset:  (11571, 128, 128, 3) (11571, 4)


In [4]:
#train second split
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)
model_dssm2 = Model(inputs=base_model.input, outputs=predictions)

model_dssm2.compile(optimizer=Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics=['acc'])

history2 = model_dssm2.fit(X_sm, Y_sm, validation_data=(testX, testY), epochs= 25, batch_size = 256)

W0805 03:34:47.951003 139935769405184 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 03:34:47.967174 139935769405184 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 03:34:47.973263 139935769405184 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0805 03:34:47.998947 139935769405184 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0805 03:34:47.999760

Train on 46284 samples, validate on 11571 samples
Epoch 1/25
46284/46284 [==============================] - 223s 5ms/step - loss: 0.5458 - acc: 0.7692 - val_loss: 0.2748 - val_acc: 0.9152
Epoch 2/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.1518 - acc: 0.9440 - val_loss: 0.2842 - val_acc: 0.9030
Epoch 3/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.0502 - acc: 0.9830 - val_loss: 0.2906 - val_acc: 0.9240
Epoch 4/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.0313 - acc: 0.9896 - val_loss: 0.3301 - val_acc: 0.9243
Epoch 5/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.0245 - acc: 0.9914 - val_loss: 0.3965 - val_acc: 0.9067
Epoch 6/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.0248 - acc: 0.9916 - val_loss: 0.4218 - val_acc: 0.9054
Epoch 7/25
46284/46284 [==============================] - 203s 4ms/step - loss: 0.0177 - acc: 0.9941 - val_loss: 0.5888 - 

In [5]:
#model predictions
predictions = model_dssm2.predict(testX)
predY = np.argmax(predictions, axis=1)
testY = np.argmax(testY, axis=1)
#Results #2
eval(testY, predY)

Accuracy: 0.922046
Precision: 0.405605
Recall: 0.344827
F1 score: 0.353959
Confusion Matrix:
 [[10549   356    59     0]
 [  307    95    24     3]
 [   98    28    24     0]
 [   18     4     5     1]]


In [ ]:
#Total Road Accidents - Third Split

In [3]:
#load data
trainX, trainY, testX, testY = load_saved_dataset('total_acc.npz')
print(Counter(trainY), Counter(testY))
#find average number of instances
m = round(sum(Counter(trainY).values())/4)
print('Mean: ', m)
#implement SMOTE on minority classes
trainX = trainX.reshape(trainX.shape[0], -1)
sm = SMOTE({1:m, 2:m, 3:m})
X_sm, Y_sm = sm.fit_sample(trainX, trainY)
print('After SMOTE: ', X_sm.shape, Y_sm.shape, Counter(Y_sm))
#split out majority set from dataset
ds_maj = []
X_tmp = []
Y_tmp = []
for i in range(len(X_sm)):
    target = Y_sm[i]
    var = X_sm[i]
    if target == 0:
        ds_maj.append(var)
    if target == 1:
        X_tmp.append(var)
        Y_tmp.append(1)
    if target == 2:
        X_tmp.append(var)
        Y_tmp.append(2)
    if target == 3:
        X_tmp.append(var)
        Y_tmp.append(3)

X_tmp = np.asarray(X_tmp)
Y_tmp = np.asarray(Y_tmp)
ds_maj = np.asarray(ds_maj)
print('After undersampling Majority: ', X_tmp.shape, Y_tmp.shape, ds_maj.shape)
#undersample majority set to m number of instances
ds_maj = shuffle(ds_maj, n_samples=m)
#generate target set of 0's for undersampled majority set
ds_maj_y = np.repeat(0, len(ds_maj))
#combine datasets
X1_tmp = np.concatenate([X_tmp, ds_maj])
Y1_tmp = np.concatenate([Y_tmp, ds_maj_y])
#shuffle combined dataset
X_sm, Y_sm = shuffle(X1_tmp, Y1_tmp, random_state=1)
print('Final Class Counter: ', Counter(Y_sm))
#prepare data for training
X_sm = X_sm.reshape(X_sm.shape[0], 128, 128, 3)
Y_sm = to_categorical(Y_sm)
testY = to_categorical(testY)
print('Final training dataset: ', X_sm.shape, Y_sm.shape)
print('Final validation dataset: ', testX.shape, testY.shape)

(46283, 128, 128, 3) (46283,) (11571, 128, 128, 3) (11571,)
Counter({0: 43941, 1: 1634, 2: 601, 3: 107}) Counter({0: 10996, 1: 415, 2: 131, 3: 29})
Mean:  11571
After SMOTE:  (78654, 49152) (78654,) Counter({0: 43941, 1: 11571, 2: 11571, 3: 11571})
After undersampling Majority:  (34713, 49152) (34713,) (43941, 49152)
Final Class Counter:  Counter({0: 11571, 1: 11571, 2: 11571, 3: 11571})
Final training dataset:  (46284, 128, 128, 3) (46284, 4)
Final validation dataset:  (11571, 128, 128, 3) (11571, 4)


In [4]:
#train third split
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)
model_dssm3 = Model(inputs=base_model.input, outputs=predictions)

model_dssm3.compile(optimizer=Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics=['acc'])

history3 = model_dssm3.fit(X_sm, Y_sm, validation_data=(testX, testY), epochs= 25, batch_size = 256)

W0804 04:09:38.751120 140070680270592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 04:09:38.770299 140070680270592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 04:09:38.776309 140070680270592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0804 04:09:38.800039 140070680270592 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0804 04:09:38.800818

Train on 46284 samples, validate on 11571 samples
Epoch 1/25
46284/46284 [==============================] - 234s 5ms/step - loss: 0.5583 - acc: 0.7652 - val_loss: 0.2164 - val_acc: 0.9365
Epoch 2/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.1496 - acc: 0.9443 - val_loss: 0.2315 - val_acc: 0.9283
Epoch 3/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.0523 - acc: 0.9827 - val_loss: 0.3757 - val_acc: 0.9008
Epoch 4/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.0324 - acc: 0.9891 - val_loss: 0.4618 - val_acc: 0.8870
Epoch 5/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.0270 - acc: 0.9909 - val_loss: 0.5224 - val_acc: 0.8671
Epoch 6/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.0198 - acc: 0.9934 - val_loss: 0.5508 - val_acc: 0.8924
Epoch 7/25
46284/46284 [==============================] - 212s 5ms/step - loss: 0.0226 - acc: 0.9924 - val_loss: 0.4014 - 

In [9]:
#model predictions
predictions = model_dssm3.predict(testX)
predY = np.argmax(predictions, axis=1)
testY = np.argmax(testY, axis=1)
#Results #3
eval(testY, predY)

Accuracy: 0.910293
Precision: 0.436648
Recall: 0.359432
F1 score: 0.354095
Confusion Matrix:
 [[10374   596    25     1]
 [  257   148     9     1]
 [   67    54     9     1]
 [    9    15     3     2]]
